## NIM Agent Blueprint for Generative Virtual Screening in Drug Discovery
This example notebook demonstrates how to connect BioNeMo NIMs to carry out a few key steps of a virtual screening workflow. Importantly, these steps are powered by highly performant AI models in each category: AlphaFold2 for folding, MolMIM for molecular generation, and DiffDock for protein-ligand docking.

Below, we illustrate this workflow using an example protein and example molecule of interest, the SARS-CoV-2 main protease and Nirmatrelvir, however, the user is free to define any protein and molecule of their choosing.

All of these capabilities are enabled by NVIDIA NIM and NVIDIA NIM Blueprints. For more details, please visit [NVIDIA NIM Blueprints](https://build.nvidia.com/nim/blueprints). 

For developers, this project is also compatible with [NVIDIA AI Workbench](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/workbench/). 

### BioNeMo Configurations
Before you begin, please set the NGC_CLI_API_KEY environment variable to a personal run key for your NGC Org and Team before running docker compose. Then, you can spin the NIMs up using the following docker command from the same directory as the `docker-compose.yaml`:

`docker compose up`

If using [NVIDIA AI Workbench](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/workbench/) to run this blueprint, set the NGC_CLI_API_KEY environment variable under ``Environment`` > ``Secrets`` to a personal run key for your NGC Org and Team. Then, select **Start** under ``Environment`` > ``Compose``. 

These containers may take a few minutes to spin up and initialize. 

In [ ]:
import requests
import os

if os.environ.get("AI_WORKBENCH_FLAG") == "true":   # AI_WORKBENCH_FLAG is set to 'true' by AI Workbench
    AF2_HOST = 'http://alphafold:8000'
    DIFFDOCK_HOST = 'http://diffdock:8000'
    MOLMIM_HOST = 'http://molmim:8000'
else: 
    AF2_HOST = 'http://localhost:8081'
    DIFFDOCK_HOST = 'http://localhost:8082'
    MOLMIM_HOST = 'http://localhost:8083'

### Protein Folding with AlphaFold2
Once a protein target of interest has been identified, the first step of this virtual screening demonstration is to generate a plausible structure of that protein. We do so by using AlphaFold2, a generative, transformer-based model that translates protein sequences into protein structures.

To demonstrate this part of our workflow, we begin with an example protein. Here, we choose the SARS-CoV-2 main protease as our starting sequence. We query the AlphaFold2 endpoint with this sequence and the model returns the predicted protein structure.

Please note that this step may take 15~20 minutes to be finished (depending on the GPU type), which is much slower than the other two inference steps that usually requires less than a minute

In [ ]:
# SARS CoV-2 main protease sequence
protein = "SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ"

In [ ]:
# Query AlphaFold2 with sequence above
af2_response = requests.post(
    f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence',
    json={
        'sequence': protein,
        'databases': ['uniref90', 'mgnify', 'small_bfd'],
        'msa_algorithm': 'jackhmmer',
        'e_value': 0.0001,
        'bit_score': -1, # -1 means to fallback to the e-value
        'msa_iterations': 1,
        'relax_prediction': True,
    }).json()

In [ ]:
# Receive protein structure for SARS CoV-2 protease
folded_protein = af2_response[0]

In [ ]:
print(folded_protein[:483])

### Molecular Generation with MolMIM
The next step in our workflow is generating molecules with optimized chemical properties starting from a seed molecule of interest. Here, molecular generation is powered by MolMIM, an LLM-inspired model aimed at generating and optimizing molecules according to user-defined objectives. The "MIM" part of MolMIM stands for Mutual Information Machine, which describes the mutual-information-based loss used to preserve chemical similarity in the model's latent space.

Here, we begin with Nirmatrelvir, an active component of the Covid treatment Paxlovid, aimed at targeting the SARS-CoV-2 main protease. By using this molecule as the input to MolMIM, the model will return 5 generated molecules with the highest chemical similarity to MolMIM. The user is able to specify the number of generated molecules to return when querying the MolMIM NIM.

Additionally, the user is able to specify chemical properties to optimize for. In this example, we have chosen to optimize the Quantitative Estimate of Drug-Likeness (QED) score, to produce molecules with favorable pharmacokinetic properties.

Note especially that here we're using the `/generate` endpoint of the MolMIM NIM.  But MolMIM was designed for controlled generation with user-defined oracles.  For this type of application you will want to call the `/decode` endpoint.  See the [documentation](https://docs.nvidia.com/nim/bionemo/molmim/latest/overview.html#decode) and [example notebook](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/cma_custom_oracles.ipynb) for additional information about using user-defined oracles.

In [ ]:
# Nirmatrelvir
molecule = "CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C"

In [ ]:
molmim_response = requests.post(
    f'{MOLMIM_HOST}/generate',
    json={
        'smi': molecule,
        'num_molecules': 5,
        'algorithm': 'CMA-ES',
        'property_name': 'QED',
        'min_similarity': 0.7, # Ignored if algorithm is not "CMA-ES".
        'iterations': 10,
    }).json()

In [ ]:
generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])

### Protein-Ligand Docking with DiffDock

After obtaining the molecules with optimized QED scores, we can predict their binding poses to the receptor target. Here, we apply DiffDock, a state-of-the-art generative model that predicts the 3D structure of a protein-ligand complex, to find out the best (most probable) binding poses. A highlighted feature from DiffDock is that a presumed binding pocket, which usually can be characterized only from experimental 3D structures, is not needed (a.k.a., blind-docking). This feature is very useful for AI folded protein structures, as it is able to locate all regions on the protein surface to be bound by drug molecules, providing ingishts for further downstream investigations.

The optimized DiffDock also provides the batch-docking function, by which we can concatenate multiple molecules into one request of docking, each of them will be also sampled for mulitple poses (i.e., num_poses=10 in this example). In the output, the predicted docking poses for each molecule is sorted by a confidence score that inferenced from a confidence model.

In [ ]:
diffdock_response = requests.post(
    f'{DIFFDOCK_HOST}/molecular-docking/diffdock/generate',
    json={
        'protein': folded_protein,
        'ligand': generated_ligands,
        'ligand_file_type': 'txt',
        'num_poses': 10,
        'time_divisions': 20,
        'num_steps': 18,
    }).json()

In [ ]:
# Print the top1 poses for each generated molecule
for i in range(len(diffdock_response['ligand_positions'])):
    print("*" * 80)
    print(diffdock_response['ligand_positions'][i][0])

In this workflow, we illustrate the ability of BioNeMo NIMs to work in concert to generate meaningful predictions in a small virtual screening workflow. We hope this underscores to the user how easy the tools are to query and assimilate, and how flexible a workflow of this sort can be.